In [1]:
# From https://ethereum.stackexchange.com/questions/111254/how-to-call-brownie-run-with-extra-script-parameters

import brownie
target = 'development'
if brownie.network.is_connected():
    brownie.network.disconnect(target)
brownie.network.connect(target)
try:
    project = brownie.project.load('/data/tutorial_brownie/hello_world_contract')
except brownie.exceptions.ProjectAlreadyLoaded:
    pass

Attached to local RPC client listening at '127.0.0.1:8545'...


In [26]:
!more proj_files/hello_world_contract.sol

pragma solidity ^0.5.11;

contract SimpleContract {
     uint value;
     function setValue(uint _value) external {value = _value;}
     function getValue() external view returns(uint){return value;}       
}


In [25]:
# Get the SimpleContract
contract = project.SimpleContract
print(contract)

In [3]:
# These are the same that Ganache creates, since we are attached to Ganache local blockchain.
list(brownie.accounts)

[<Account '0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1'>,
 <Account '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0'>,
 <Account '0x22d491Bde2303f2f43325b2108D26f1eAbA1e32b'>,
 <Account '0xE11BA2b4D45Eaed5996Cd0823791E0C93114882d'>,
 <Account '0xd03ea8624C8C5987235048901fB614fDcA89b117'>,
 <Account '0x95cED938F7991cd0dFcb48F0a06a40FA1aF46EBC'>,
 <Account '0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9'>,
 <Account '0x28a8746e75304c0780E011BEd21C72cD78cd535E'>,
 <Account '0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E'>,
 <Account '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e'>]

In [9]:
# Get the first account.
accounts = brownie.accounts
account = accounts[0]
print(account)

#dir(account)

0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1


In [22]:
# The balance is the same as reported by Ganache (1000 ETH in Wei)
print(accounts[0].balance())
print(accounts[1].balance())

print("total=", accounts[0].balance() + accounts[1].balance())

973981846639966316000
1021967099660000000000
total= 1995948946299966316000


In [11]:
# From https://stackoverflow.com/questions/71341281/awaiting-transaction-in-the-mempool

if True:
    # In dev mode.
    from brownie.network import gas_price
    from brownie.network.gas.strategies import LinearScalingStrategy

    gas_strategy = LinearScalingStrategy("60 gwei", "70 gwei", 1.1)

    #if network.show_active() == "development":
    gas_price(gas_strategy)

In [12]:
# Transfer 10 ether from account 0 to 1.
accounts[0].transfer(accounts[1], "10 ether")

Transaction sent: 0xa9d29d6ac93599393f0cf3e3587e679904148051fa488ac05db397683cf1d0ca
  Gas price: 60.0 gwei   Gas limit: 30000000   Nonce: 0
  Transaction confirmed   Block: 1   Gas used: 21000 (0.07%)



<Transaction '0xa9d29d6ac93599393f0cf3e3587e679904148051fa488ac05db397683cf1d0ca'>

In [23]:
# Note that some ether was spent in gas.
print(accounts[0].balance())
print(accounts[1].balance())

print("total=", accounts[0].balance() + accounts[1].balance())

973981846639966316000
1021967099660000000000
total= 1995948946299966316000


In [16]:
# List contracts deployed by Brownie in this session.
list(contract)

[]

In [17]:
# Report the ABI of the hello_world_contract.sol
contract.abi

[{'constant': True,
  'inputs': [],
  'name': 'getValue',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'payable': False,
  'stateMutability': 'view',
  'type': 'function'},
 {'constant': False,
  'inputs': [{'internalType': 'uint256', 'name': '_value', 'type': 'uint256'}],
  'name': 'setValue',
  'outputs': [],
  'payable': False,
  'stateMutability': 'nonpayable',
  'type': 'function'}]

In [18]:
# Deploy the contract from account 0.
contract_tmp = accounts[0].deploy(contract)

print(contract_tmp)

Transaction sent: 0x24a47c91acd604a169e533fcbbc1004d71b17b89dd10a5fa7f76c53821260874
  Gas price: 60.0 gwei   Gas limit: 30000000   Nonce: 1
  SimpleContract.constructor confirmed   Block: 2   Gas used: 90351 (0.30%)
  SimpleContract deployed at: 0x5b1869D9A4C187F2EAa108f3062412ecf0526b24

0x5b1869D9A4C187F2EAa108f3062412ecf0526b24


In [19]:
# Brownie is keeping track of the contracts.
list(contract)

[<SimpleContract Contract '0x5b1869D9A4C187F2EAa108f3062412ecf0526b24'>]

In [20]:
contract.signatures

{'getValue': '0x20965255', 'setValue': '0x55241077'}

In [21]:
# Execute a transaction on the contract.
print("getValue before=", contract[0].getValue())

# Set the value.
tx = contract[0].setValue(10000)
print(tx)

tx.info()

# After the contract the value.
print("getValue after=", contract[0].getValue())

getValue before= 0
Transaction sent: 0x697c99f4b0674057072a32ed5b7de8eb82050c680a263962c79e32b286aea00c
  Gas price: 60.0 gwei   Gas limit: 30000000   Nonce: 2
  SimpleContract.setValue confirmed   Block: 3   Gas used: 43505 (0.15%)

<Transaction '0x697c99f4b0674057072a32ed5b7de8eb82050c680a263962c79e32b286aea00c'>
Transaction was Mined 
---------------------
Tx Hash: 0x697c99f4b0674057072a32ed5b7de8eb82050c680a263962c79e32b286aea00c
From: 0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1
To: 0x5b1869D9A4C187F2EAa108f3062412ecf0526b24
Value: 0
Function: SimpleContract.setValue
Block: 3
Gas Used: 43505 / 30000000 (0.1%)

getValue after= 10000
